In [1]:
"""_summary_
df_vehicle  : vehicle spec csv file
                assumptions:
                + year affects it ? no becuase not a primary key
df_0        : parquet file 1 for driving data - raw
df_0_1      : df_0 with date and time seperated into seperate columns 
                assumptions: 
                + duplicate times / vehicle spec ids - do a distinct at the start (if necessary after confirming)
                + trip duration : max_time - min_time per vehicle, on that 
                    + vehicle was running continuously without stopping 
                    + can we figure out if it did stop? exclude velocity = 0/null data
                + distance travelled: distance between the lat/long of max_time and min_time
                    + this is the displacement not the distance
                    + to improve : use a lag/lead to calculate the displacement between each timestamp update per vehicle and sum up total distance travelled 




"""

'_summary_\ndf_vehicle  : vehicle spec csv file\n                assumptions:\n                + year affects it ? no becuase not a primary key\ndf_0        : parquet file 1 for driving data - raw\ndf_0_1      : df_0 with date and time seperated into seperate columns \n                assumptions: \n                + duplicate times / vehicle spec ids - do a distinct at the start (if necessary after confirming)\n                + trip duration : max_time - min_time per vehicle, on that \n                    + vehicle was running continuously without stopping \n                    + can we figure out if it did stop? exclude velocity = 0/null data\n                + distance travelled: distance between the lat/long of max_time and min_time\n                    + this is the displacement not the distance\n                    + to improve : use a lag/lead to calculate the displacement between each timestamp update per vehicle and sum up total distance travelled \n\n\n\n\n'

In [2]:
import pandas as pd
import pyarrow.parquet as pa
from pathlib import Path
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window





In [3]:
data_path = '/Users/athulparvelikudy/Personal/ACRTA/tech-test-data/supporting-data'
file_name_0 = 'drive/part-00000-tid-4109877695252048813-a3139a95-1807-419c-af03-4877385b4c8c-11-1-c000.snappy.parquet'
file_name_1 = 'drive/part-00001-tid-4109877695252048813-a3139a95-1807-419c-af03-4877385b4c8c-12-1-c000.snappy.parquet'
file_name_2 = 'drive/part-00002-tid-4109877695252048813-a3139a95-1807-419c-af03-4877385b4c8c-13-1-c000.snappy.parquet'
file_name_3 = 'drive/part-00003-tid-4109877695252048813-a3139a95-1807-419c-af03-4877385b4c8c-14-1-c000.snappy.parquet'
file_name_4 = 'drive/part-00004-tid-4109877695252048813-a3139a95-1807-419c-af03-4877385b4c8c-15-1-c000.snappy.parquet'
vehicle_file = 'vehicle.csv'

vehicle_csv = str(Path(data_path) / vehicle_file)

full_file_path0 = str(Path(data_path) / file_name_0)
full_file_path1 = str(Path(data_path) / file_name_1)
full_file_path2 = str(Path(data_path) / file_name_2)
full_file_path3 = str(Path(data_path) / file_name_3)
full_file_path4 = str(Path(data_path) / file_name_4)



In [4]:


spark = (
    SparkSession.builder
    .appName("ReadLocalParquet")
    .getOrCreate()
)


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/27 15:19:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark.version

'4.1.0'

In [6]:
df_0 = spark.read.parquet(full_file_path0)


In [7]:
df_0.printSchema()
df_0.count()


root
 |-- trip_id: string (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- vehicle_spec_id: long (nullable = true)
 |-- engine_coolant_temp: double (nullable = true)
 |-- eng_load: double (nullable = true)
 |-- fuel_level: double (nullable = true)
 |-- iat: double (nullable = true)
 |-- rpm: double (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- velocity: double (nullable = true)



1843506

In [8]:
df_vehicle = (
    spark.read
    .option("header", "true") 
    .option("inferSchema", "true") 
    .csv(vehicle_csv)
)

df_vehicle.printSchema()
df_vehicle.count()


root
 |-- vehicle_spec_id: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- make: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- drivetrain: integer (nullable = true)
 |-- max_torque: integer (nullable = true)
 |-- max_horsepower: integer (nullable = true)
 |-- max_horsepower_rpm: integer (nullable = true)
 |-- max_torque_rpm: integer (nullable = true)
 |-- engine_displacement: double (nullable = true)
 |-- fuel_type: integer (nullable = true)
 |-- fuel_tank_capacity: integer (nullable = true)
 |-- fuel_economy_city: double (nullable = true)
 |-- fuel_economy_highway: double (nullable = true)
 |-- cylinders: integer (nullable = true)
 |-- forced_induction: string (nullable = true)
 |-- device_generation: integer (nullable = true)



7

In [9]:
print("Parquet file (10 rows):")
df_0.show(10, truncate=False)

print("CSV file (10 rows):")
df_vehicle.show(10, truncate=False)


Parquet file (10 rows):
+--------------------------------+-------------------+---------------+-------------------+--------+----------+------+-------+------------------+-------------------+--------+
|trip_id                         |datetime           |vehicle_spec_id|engine_coolant_temp|eng_load|fuel_level|iat   |rpm    |lat               |long               |velocity|
+--------------------------------+-------------------+---------------+-------------------+--------+----------+------+-------+------------------+-------------------+--------+
|1e20465533c545f98332ff14d5a0af22|2017-01-06 14:41:24|1000502        |177.8              |210.65  |76.0      |99.63 |2110.55|31.1275           |-96.1875           |61.01   |
|7ddd6571ac1f42959a07de2b599edde8|2017-01-07 14:21:24|1000511        |174.0              |210.79  |143.07    |69.0  |2053.72|30.79416666666667 |-85.3125           |58.04   |
|afe9ae193f1544398c289a0cb3ff611e|2017-02-14 08:21:22|1000501        |96.0               |210.18  |75.0   

In [10]:
df_0_1 = (
    df_0
    .withColumn("date", F.to_date("datetime"))
    .withColumn("time", F.date_format("datetime", "HH:mm:ss"))
)

In [ ]:
input_dt = "2017-01-06"
# TODO: make a check to see if the input is in the correct format before processing

df_2017_01_06 = df_0_1.filter(F.col("date") == F.lit(input_dt))

df_2017_01_06.count() # around 300k

# df_2017_01_06.show(20, truncate=False)

# df_2017_01_06.orderBy(F.col("time").desc()).show(20, truncate=False)

df_2017_01_06.orderBy("time", ascending=False).limit(5).show(truncate=False)

df_joined = (
    df_0_1
    .filter(F.col("date") == F.to_date(F.lit(input_dt)))
    .join(df_vehicle, on="vehicle_spec_id", how="left")
).limit(5).show()

+--------------------------------+-------------------+---------------+-------------------+--------+----------+------+-------+-----------------+------------------+--------+----------+--------+
|trip_id                         |datetime           |vehicle_spec_id|engine_coolant_temp|eng_load|fuel_level|iat   |rpm    |lat              |long              |velocity|date      |time    |
+--------------------------------+-------------------+---------------+-------------------+--------+----------+------+-------+-----------------+------------------+--------+----------+--------+
|3f016fd4116642048584a772e191b656|2017-01-06 23:59:59|1000503        |147.8              |206.22  |97.0      |51.0  |1925.99|29.9375          |-92.06277777777777|91.88   |2017-01-06|23:59:59|
|258c58632cd946ae994c631fdef786ad|2017-01-06 23:59:59|1000505        |154.97             |189.0   |109.0     |81.0  |2097.02|31.0625          |-86.43777777777778|68.27   |2017-01-06|23:59:59|
|a810e2af97fd4652a2469796db67d2ce|2017-0

25/12/27 15:19:57 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [18]:


df_trip_duration_1 = (
    df_2017_01_06
    .groupBy("vehicle_spec_id", "trip_id")
    .agg(
        F.min("datetime").alias("min_datetime"),
        F.max("datetime").alias("max_datetime"),
    )
    .withColumn("trip_duration_seconds", F.unix_timestamp("max_datetime") - F.unix_timestamp("min_datetime"))
    .select( "trip_id", "vehicle_spec_id", "trip_duration_seconds")
)

df_trip_duration_2 = (
    df_2017_01_06
    .groupBy("vehicle_spec_id")
    .agg(
        F.min("datetime").alias("min_datetime"),
        F.max("datetime").alias("max_datetime"),
    )
    .withColumn("trip_duration_seconds", F.unix_timestamp("max_datetime") - F.unix_timestamp("min_datetime"))
    .select( "vehicle_spec_id", "trip_duration_seconds")
)


# df_trip_duration.show(20, truncate=False)
vehicle_cnt = df_2017_01_06.select("vehicle_spec_id").distinct().count()
print(f"df_2017_01_06 vehicles: {vehicle_cnt}")

trip_cnt = df_2017_01_06.select("vehicle_spec_id", "trip_id").distinct().count()
print(f"df_2017_01_06 trips: {trip_cnt}")

print(f'df_trip_duration_1: {df_trip_duration_1.count()}')
print(f'df_trip_duration_2: {df_trip_duration_2.count()}')

df_trip_duration_1.select("vehicle_spec_id", "trip_id").distinct().count()
df_trip_duration_2.select("vehicle_spec_id").distinct().count()


df_2017_01_06 vehicles: 20
df_2017_01_06 trips: 336
df_trip_duration_1: 336
df_trip_duration_2: 20


20

In [22]:
df_duration = (
    df_2017_01_06
    .groupBy("vehicle_spec_id")
    .agg(
        F.min("datetime").alias("min_datetime"),
        F.max("datetime").alias("max_datetime"),
    )
    .withColumn("trip_duration_seconds", F.unix_timestamp("max_datetime") - F.unix_timestamp("min_datetime"))
    .select( "vehicle_spec_id", "trip_duration_seconds")
)

w = Window.partitionBy("vehicle_spec_id").orderBy(F.col("datetime").asc())

df_min_trip = (
    df_2017_01_06
    .withColumn("rn", F.row_number().over(w))
    .filter(F.col("rn") == 1)
    .select(
        "vehicle_spec_id",
        F.col("trip_id").alias("min_time_trip_id")
    )
)

df_trip_duration = (
    df_duration
    .join(df_min_trip, on="vehicle_spec_id", how="left")
    .select("vehicle_spec_id", "min_time_trip_id", "trip_duration_seconds")
)

df_trip_duration.count()


20

In [29]:
w_asc = Window.partitionBy("vehicle_spec_id").orderBy(F.col("datetime").asc())
w_desc = Window.partitionBy("vehicle_spec_id").orderBy(F.col("datetime").desc())


df_start = (
    df_2017_01_06
    .withColumn("rn", F.row_number().over(w_asc))
    .filter(F.col("rn") == 1)
    .select(
        "vehicle_spec_id",
        F.col("trip_id").alias("start_trip_id"),
        F.col("datetime").alias("start_datetime"),
        F.col("lat").alias("start_lat"),
        F.col("long").alias("start_long"),
    )
)

df_end = (
    df_2017_01_06
    .withColumn("rn", F.row_number().over(w_desc))
    .filter(F.col("rn") == 1)
    .select(
        "vehicle_spec_id",
        F.col("trip_id").alias("end_trip_id"),
        F.col("datetime").alias("end_datetime"),
        F.col("lat").alias("end_lat"),
        F.col("long").alias("end_long"),
    )
)

df_start_end = df_start.join(df_end, on="vehicle_spec_id", how="inner")

df_start_end.show(5, truncate=False)

df_start_end = (
    df_start.alias("s")
    .join(df_end.alias("e"), on="vehicle_spec_id", how="inner")
   .select(
        F.col("vehicle_spec_id"),
        F.col("s.start_trip_id"),
        F.col("e.end_trip_id"),
        F.col("s.start_datetime"),
        F.col("e.end_datetime"),
        F.col("s.start_lat"),
        F.col("s.start_long"),
        F.col("e.end_lat"),
        F.col("e.end_long"),
    )
    .withColumn(
        "trip_duration_seconds",
        F.unix_timestamp("end_datetime") - F.unix_timestamp("start_datetime")
    )
    # haversine distance in km
    .withColumn("start_lat_r", F.radians("start_lat"))
    .withColumn("end_lat_r", F.radians("end_lat"))
    .withColumn("dlat", F.radians(F.col("end_lat") - F.col("start_lat")))
    .withColumn("dlon", F.radians(F.col("end_long") - F.col("start_long")))
    .withColumn(
        "a",
        F.pow(F.sin(F.col("dlat") / 2), 2)
        + F.cos("start_lat_r") * F.cos("end_lat_r") * F.pow(F.sin(F.col("dlon") / 2), 2)
    )
    .withColumn("c", 2 * F.atan2(F.sqrt("a"), F.sqrt(1 - F.col("a"))))
    .withColumn("distance_km", F.lit(6371.0) * F.col("c"))
    .drop("start_lat_r", "end_lat_r", "dlat", "dlon", "a", "c")
)

df_start_end.show(5, truncate=False)




+---------------+--------------------------------+-------------------+------------------+------------------+--------------------------------+-------------------+-----------------+------------------+
|vehicle_spec_id|start_trip_id                   |start_datetime     |start_lat         |start_long        |end_trip_id                     |end_datetime       |end_lat          |end_long          |
+---------------+--------------------------------+-------------------+------------------+------------------+--------------------------------+-------------------+-----------------+------------------+
|1000500        |62153c1ba9a340c79fa75f92d19d261b|2017-01-06 14:00:06|30.439166666666665|-96.1875          |72de8a5720ae4183881f062ba60449eb|2017-01-06 23:58:21|33.035           |-88.4375          |
|1000501        |0244f7bc747b41fba6fcd75444736621|2017-01-06 15:00:01|29.68777777777778 |-101.1875         |9d2d32395bf54291a83bdd19e0c4e189|2017-01-06 23:59:58|31.6875          |-90.56305555555555|
|1000

In [30]:
df_final = (
    df_start_end
    .join(
        df_vehicle.select("vehicle_spec_id", "make", "model"),
        on="vehicle_spec_id",
        how="left"
    )
)

df_final.select(
    "start_trip_id",
    "end_datetime",
    "trip_duration_seconds",
    "distance_km",
    "make",
    "model"
).show(10, truncate=False)



+--------------------------------+-------------------+---------------------+------------------+----------+-----------+
|start_trip_id                   |end_datetime       |trip_duration_seconds|distance_km       |make      |model      |
+--------------------------------+-------------------+---------------------+------------------+----------+-----------+
|62153c1ba9a340c79fa75f92d19d261b|2017-01-06 23:58:21|35895                |787.4240369143236 |Honda     |Civic      |
|0244f7bc747b41fba6fcd75444736621|2017-01-06 23:59:58|32397                |1039.5202265482276|Jeep      |Compass    |
|83d2d3ceee6843ec8001e5b585bd426e|2017-01-06 23:59:58|68393                |1081.8225180749207|Hyundai   |Creta      |
|1e93dd4cdd4e4b4b8b122a0aa72191a8|2017-01-06 23:59:59|50393                |471.74513507174544|Skoda     |Superb     |
|0c538955f73e42d08f29565cdd6d3bd8|2017-01-06 23:59:59|68393                |830.8667528254012 |Volkswagen|Passat     |
|3e59754036674a34b45d05048e080a3c|2017-01-06 23:

In [12]:
"""df0 = pa.read_table(full_file_path0)
print(df0.shape)
df0.slice(0, 10).to_pandas() 

df1 = pa.read_table(full_file_path1)
print(df1.shape)
df1.slice(0, 10).to_pandas()

df2 = pa.read_table(full_file_path2)
print(df2.shape)
df2.slice(0, 10).to_pandas()

df3 = pa.read_table(full_file_path3)
print(df3.shape)
df3.slice(0, 10).to_pandas()

df4 = pa.read_table(full_file_path4)
print(df4.shape)
df4.slice(0, 10).to_pandas()
"""

'df0 = pa.read_table(full_file_path0)\nprint(df0.shape)\ndf0.slice(0, 10).to_pandas() \n\ndf1 = pa.read_table(full_file_path1)\nprint(df1.shape)\ndf1.slice(0, 10).to_pandas()\n\ndf2 = pa.read_table(full_file_path2)\nprint(df2.shape)\ndf2.slice(0, 10).to_pandas()\n\ndf3 = pa.read_table(full_file_path3)\nprint(df3.shape)\ndf3.slice(0, 10).to_pandas()\n\ndf4 = pa.read_table(full_file_path4)\nprint(df4.shape)\ndf4.slice(0, 10).to_pandas()\n'